In [ ]:
# Import the neccessary modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = 'https://frenzy86.s3.eu-west-2.amazonaws.com/fav/tecno/titanic.csv'

df = pd.read_csv(path, sep='\t')
#df = pd.read_csv(path,)
df

In [ ]:
plt.figure(figsize=(7,5))
sns.heatmap(df.isnull(),cbar=False)     #toglie barra colori

In [ ]:
#operazioni da fare in EDA

df.info()
df.describe().T
df['column_name'].value_counts()
df['column_name'].value_counts(normalize=True)
df.groupby('column_name').count()
df.isnull().sum()
df["column_orig"] = df["column_changed"]     #cambio nome colonna
df["species"].unique()
df.dropna()              #droppare elementi che non servono


In [ ]:
# To replace missing values with interpolated values
df['chosen_column_to_fill'] = df['chosen_column_to_fill'].interpolate()


In [ ]:
df = df.drop(['Name', 'Ticket', 'Cabin','PassengerId'], axis=1)       #droppare colonne non necessarie
df

VISUALIZZAZIONE

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('inserire il titolo')
sns.countplot(x='column_name', hue="column_name", data=df)
sns.countplot(x='selected_value',data=df,hue='selected_value')

In [ ]:
df =  pd.get_dummies(df, columns=['Embarked','Sex'],dtype=int, drop_first=True)
df

In [ ]:
# generate binary values using get_dummies
bridge_df = pd.DataFrame(df, columns=['Bridge_Types'])
dum_df = pd.get_dummies(bridge_df, columns=["Bridge_Types"],dtype=int,drop_first=True)#, prefix=["Type_is"] )
dum_df

In [ ]:
#definire features e target, ad esempio:

X_ = df.iloc[:,0:-1]      #dati non notmalizzati
y = df.iloc[:,-1]

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
standardized_data = sc.fit_transform(X_)

X = standardized_data

In [ ]:
X.shape
y.shape      #controllare X, y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=667,
                                                    stratify=y
                                                    )

In [ ]:
X_test.shape

In [ ]:
#EX 1:

from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test, y_test)

In [ ]:
#EX 2:

from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=200)
rf_clf.fit(X_train, y_train)
rf_clf.score(X_test, y_test) # accuracy

PROCEDIMENTO CON PIU' CLASSIFICATORI INSIEME

In [ ]:
classifiers = [DecisionTreeClassifier(max_depth=4),
               RandomForestClassifier(n_estimators=200,random_state=667),
               GradientBoostingClassifier(),
               GradientBoostingClassifier(n_estimators=50),
               LogisticRegression(max_iter=1000),
               GaussianNB(),
               KNeighborsClassifier(n_neighbors=2),
               SVC(probability=True,kernel='rbf'),
               XGBClassifier()
               ]

In [ ]:
len(classifiers)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=667,
                                                    )


for cls in classifiers:
    cls.fit(X_train, y_train)
    cls.predict(X_test)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15,10))

labels = ['morti','sopravv']
for cls, ax in zip(classifiers, axes.flatten()):
    ConfusionMatrixDisplay.from_estimator(cls,
                                            X_test,
                                            y_test,
                                            ax=ax,
                                            cmap='Blues',
                                            display_labels=labels,
                                            )
    ax.title.set_text(type(cls).__name__)
plt.tight_layout()

In [ ]:
from sklearn.metrics import classification_report

for cls, ax in zip(classifiers, axes.flatten()):
    print(cls)
    print(classification_report(y_test,cls.predict(X_test),target_names=labels))

In [ ]:
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score

# Train the models and record the results
plt.figure(figsize=(8,7))
ax = plt.gca()
plt.plot([0,1],[0,1],'--b')

for cls in classifiers:
    model = cls.fit(X_train, y_train)
    y_score =model.predict_proba(X_test)[:, 1]
    auc_model = roc_auc_score(y_test,y_score)

    RocCurveDisplay.from_estimator(model, X_test, y_test, label=f'{cls.__class__.__name__} (AUC = {auc_model:.2f})', ax=ax)
    plt.title("Receiver Operating Characteristic (ROC) curves")
    plt.legend(loc=4)

SCEGLI MODELLO MIGLIORE

In [ ]:
model_xcb = XGBClassifier()

In [ ]:
model_xcb.fit(X_train, y_train)

In [ ]:
model_xcb.score(X_train, y_train)

In [ ]:
model_xcb.score(X_test, y_test)

In [ ]:
importance = model_xcb.feature_importances_

PIPELINE

In [ ]:
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer,OneHotEncoder,StandardScaler,FunctionTransformer

In [ ]:
## dividiamo automaticamente le features numeriche da quelle categoriche

numerical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind in ['i','f'] ]
categorical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind not in ['i','f']]

print('Numerical : ' + str(numerical_features))
print('Categorical : ' + str(categorical_features))

In [ ]:
Preprocessor = make_column_transformer(
                                        (make_pipeline(
                                        SimpleImputer(strategy = 'median'),
                                        KBinsDiscretizer(n_bins=3)), numerical_features),

                                        (make_pipeline(
                                        #FunctionTransformer(lambda x: np.where(x == 'n.d.', np.nan, x)),
                                        SimpleImputer(strategy = 'constant', fill_value = 'missing'),
                                        OneHotEncoder(categories = 'auto', handle_unknown = 'ignore',drop='first')), categorical_features)
                                        )

In [ ]:
scaler = StandardScaler()

classifier = RandomForestClassifier(bootstrap=True,
                                    max_depth=80,
                                    max_features=2,
                                    min_samples_leaf=3,
                                    min_samples_split=8,
                                    n_estimators=100
                                    )
#classifier = LogistiRegression()

In [ ]:
model_pipe = Pipeline([
                       ('Preprocessing features', Preprocessor),
                       ('Scaling and standadize data', scaler),
                       ('Classifier', classifier)
                      ])
model_pipe

In [ ]:
cross_val_score(pipe, X, y, cv=5, error_score='raise')

In [ ]:
model_pipe.fit(X, y)

In [ ]:
import joblib

joblib.dump(model_pipe, "model_svc_iris.pkl")

In [ ]:
import joblib

model_pipe = joblib.load('model_svc_iris.pkl')
print('Model loaded successfully')